In [1]:
from random import seed, randrange
from csv import reader


seed(42)


def load_csv(filepath):
  dataset = []

  with open(filepath, "r") as file:
    csv_reader = reader(file)
    for row in csv_reader:
      if row:
        dataset.append(row)

  return dataset


def str_col_to_float(dataset, col):
  for row in dataset:
    row[col] = float(row[col].strip())


def cross_validation_split(dataset, n_folds):
  dataset_folds = []
  fold_size = int(len(dataset) / n_folds)
  dataset_ = dataset.copy()

  for _ in range(n_folds):
    fold = []
    while len(fold) < fold_size:
      fold.append(dataset_.pop(randrange(len(dataset_))))
    dataset_folds.append(fold)

  return dataset_folds


def accuracy_metric(actual, predicted):
  assert len(actual) == len(predicted)

  correct_count = 0
  
  for i in range(len(actual)):
    correct_count += 1 if actual[i] == predicted[i] else 0

  return (correct_count / len(actual)) * 100


def evaluate_algorithm(dataset, algorithm, n_folds, *args):
  folds = cross_validation_split(dataset=dataset, n_folds=n_folds)
  scores = []

  for fold in folds:
    train = folds.copy()
    train.remove(fold)
    train = sum(train, [])
    test = []
    for row in fold:
      row_ = row.copy()
      row_[-1] = None
      test.append(row_)
    predicted = algorithm(train, test, *args)
    actual = list(map(lambda row: row[-1], fold))
    accuracy = accuracy_metric(actual=actual, predicted=predicted)
    scores.append(accuracy)

  return scores


def gini_index(groups, classes):
  n_instances = float(sum(list(map(lambda group: len(group), groups))))
  gini = 0.0

  for group in groups:
    size = float(len(group))
    if size != 0:
      score = 0.0
      for class_val in classes:
        proportion = [row[-1] for row in group].count(class_val) / size
        score += pow(proportion, 2)
      gini += (1.0 - score) * (size / n_instances)

  return gini


def test_split(dataset, index, value):
  left, right = [], []
  
  for row in dataset:
    if row[index] < value:
      left.append(row)
    else:
      right.append(row)
  
  return left, right


def get_split(dataset):
  class_vals = list(set(map(lambda row: row[-1], dataset)))
  split_metadata = {'index': 999, 'value': 999, 'groups': None}
  score = 999

  for index in range(len(dataset[0]) - 1):
    for row in dataset:
      groups = test_split(dataset=dataset, index=index, value=row[index])
      gini = gini_index(groups=groups, classes=class_vals)
      if gini < score:
        score = gini
        split_metadata = {'index': index, 'value': row[index], 'groups': groups}
  
  return split_metadata


def to_terminal(group):
  class_vals = list(map(lambda row: row[-1], group))
  
  return max(set(class_vals), key=class_vals.count)


def split(node, max_depth, min_size, depth):
  left, right = node['groups']
  
  del(node['groups'])
  if not left or not right:
    node['left'] = node['right'] = to_terminal(left + right)
    return
  if depth >= max_depth:
    node['left'], node['right'] = to_terminal(left), to_terminal(right)
    return
  if len(left) < min_size:
    node['left'] = to_terminal(left)
  else:
    node['left'] = get_split(dataset=left)
    split(node=node['left'], max_depth=max_depth, min_size=min_size, depth=depth + 1)
  if len(right) < min_size:
    node['right'] = to_terminal(right)
  else:
    node['right'] = get_split(dataset=right)
    split(node=node['right'], max_depth=max_depth, min_size=min_size, depth=depth + 1)


def build_tree(train, max_depth, min_size):
  root = get_split(dataset=train)

  split(node=root, max_depth=max_depth, min_size=min_size, depth=1)

  return root


def print_tree(node, depth=0):
  if isinstance(node, dict):
    print(f"{depth * ' '}[X{node['index'] + 1} < {node['value']}]")
    print_tree(node['left'], depth + 1)
    print_tree(node['right'], depth + 1)
  else:
    print(f"{depth * ' '}[{node}]")


def predict(node, row):
  if row[node['index']] < node['value']:
    if isinstance(node['left'], dict):
      return predict(node['left'], row)
    else:
      return node['left']
  else:
    if isinstance(node['right'], dict):
      return predict(node['right'], row)
    else:
      return node['right']


def decision_tree(train, test, max_depth, min_size):
  tree = build_tree(train=train, max_depth=max_depth, min_size=min_size)
  
  print_tree(tree)

  return list(map(lambda row: predict(node=tree, row=row), test))


filepath = "../datasets/data-banknote-authentication.csv"
dataset = load_csv(filepath=filepath)
for i in range(len(dataset[0])):
  str_col_to_float(dataset=dataset, col=i)
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
display(f"scores={scores}")
display(f"mean accuracy={sum(scores) / len(scores)}")

[X1 < 0.3223]
 [X2 < 8.1881]
  [X1 < -0.45062]
   [X1 < -1.8584]
    [X1 < -2.7769]
     [1.0]
     [1.0]
    [X3 < 6.9144]
     [1.0]
     [1.0]
   [X3 < 0.49839]
    [X2 < 7.0521]
     [1.0]
     [0.0]
    [X4 < 0.7336]
     [0.0]
     [1.0]
  [X1 < -4.2859]
   [X1 < -6.0598]
    [1.0]
    [1.0]
   [X1 < -1.4572]
    [X1 < -2.7143]
     [0.0]
     [0.0]
    [X1 < -1.4572]
     [0.0]
     [0.0]
 [X3 < -4.3582]
  [X1 < 4.2164]
   [X1 < 1.6408]
    [X1 < 1.1644]
     [1.0]
     [1.0]
    [1.0]
   [0.0]
  [X1 < 1.594]
   [X3 < -2.2718]
    [X2 < 7.6377]
     [1.0]
     [0.0]
    [X4 < 0.097399]
     [0.0]
     [0.0]
   [X1 < 2.0421]
    [X3 < -2.3386]
     [1.0]
     [0.0]
    [X1 < 4.1736]
     [0.0]
     [0.0]
[X1 < 0.3223]
 [X2 < 5.8974]
  [X3 < 6.2204]
   [X2 < 4.1158]
    [X3 < 4.9068]
     [1.0]
     [1.0]
    [X2 < 4.1986]
     [0.0]
     [1.0]
   [X2 < -4.6062]
    [X1 < -0.16108]
     [1.0]
     [0.0]
    [X1 < -1.7781]
     [0.0]
     [0.0]
  [X1 < -2.7419]
   [X1 < -5.4414]
  

'scores=[95.62043795620438, 97.8102189781022, 95.62043795620438, 96.71532846715328, 97.44525547445255]'

'mean accuracy=96.64233576642337'